In [14]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [15]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [16]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is sheenu and I am a AI intern")])

AIMessage(content="Hello Sheenu, it's nice to meet you!\n\nI'm glad to have an AI intern on board. What kind of work will you be doing during your internship?  \n\nAre you working on any interesting projects? I'd love to hear more about your experience as an AI intern.\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 27, 'total_tokens': 92, 'completion_time': 0.134012061, 'prompt_time': 0.00264505, 'queue_time': None, 'total_time': 0.136657111}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f75f2b2-3215-4537-8a42-3ed0d6f5c3c2-0', usage_metadata={'input_tokens': 27, 'output_tokens': 65, 'total_tokens': 92})

In [17]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is sheenu and I am a AI intern"),
        AIMessage(content="Hello Sheenu! It's nice to meet you.  \n\nWelcome to the world of AI"),
        HumanMessage(content="what's my name and profession?")
    ]
)

AIMessage(content="You told me your name is Sheenu and that you are an AI intern.  \n\nIs there anything else you'd like to tell me about yourself?  I'm eager to learn more! 😊 \n", response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 65, 'total_tokens': 111, 'completion_time': 0.09402839, 'prompt_time': 0.004531279, 'queue_time': None, 'total_time': 0.098559669}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-50c94f92-c2df-4f6c-a77d-2fed8bd5ad26-0', usage_metadata={'input_tokens': 65, 'output_tokens': 46, 'total_tokens': 111})

### MESSAGE HISTORY
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)  #now we can able to interact with out model based on message history

In [20]:
config={"configurable":{"session_id":"chat1"}}

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is sheenu and I am a Chief AI Engineer")],
    config=config
)

In [25]:
response

AIMessage(content="Hello Sheenu, it's great to meet you!  Chief AI Engineer is a fantastic title!  What kind of AI projects are you most passionate about? \n\nI'm always curious to hear about the innovative work people are doing in the field.  😊  \n", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 164, 'total_tokens': 223, 'completion_time': 0.122298854, 'prompt_time': 0.007993918, 'queue_time': None, 'total_time': 0.130292772}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2c14753-9439-43fc-8a66-62b0d3e59b14-0', usage_metadata={'input_tokens': 164, 'output_tokens': 59, 'total_tokens': 223})

In [26]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You told me your name is Sheenu! 😊  \n\nIs there anything else I can help you with? \n', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 237, 'total_tokens': 263, 'completion_time': 0.052459359, 'prompt_time': 0.040626032, 'queue_time': None, 'total_time': 0.09308539099999999}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d1d1c279-ab75-47b9-9fd1-7e4e4f00af67-0', usage_metadata={'input_tokens': 237, 'output_tokens': 26, 'total_tokens': 263})

### prompt template
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [29]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")]})

AIMessage(content="Hi Sheenu, it's nice to meet you!\n\nWhat can I do for you today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 37, 'total_tokens': 63, 'completion_time': 0.052400538, 'prompt_time': 0.002990332, 'queue_time': None, 'total_time': 0.05539087}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-81a84916-333b-4ac8-a449-1c948a59f933-0', usage_metadata={'input_tokens': 37, 'output_tokens': 26, 'total_tokens': 63})

In [30]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [31]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is sheenu")],
    config=config
)

response

AIMessage(content="Hi Sheenu, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 37, 'total_tokens': 64, 'completion_time': 0.054510978, 'prompt_time': 0.003022191, 'queue_time': None, 'total_time': 0.057533169}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6839316a-40f8-4298-b338-5565aad99602-0', usage_metadata={'input_tokens': 37, 'output_tokens': 27, 'total_tokens': 64})

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sheenu.  😄 \n\nI remember! \n\n\n\n'

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [38]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")],"language":"hindi"})
response.content

'नमस्ते शीनु! 😊 \n\nमुझे बहुत खुशी है कि आपने मुझसे मिलकर बात की। मैं आपकी मदद करने के लिए तैयार हूँ। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [39]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

input_messages_key="messages" parameter specifies the key in the input dictionary where the messages are stored. This key tells the function where to find the list of messages within the input data structure.

In [40]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते, कृष्णा! \n\nआप कैसे हैं?  😊\n'

In [41]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [42]:
response.content

'आपका नाम कृष्णा है!  \n'